In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image
import requests


asset_list = glob('/media/sj-archimedes/data/masaya_kondo/research/mllm/box_asset/list/*.pkl')
asset_list_df = pd.concat([pd.read_pickle(path) for path in asset_list])
asset_img_dir = '/media/sj-archimedes/data/masaya_kondo/research/mllm/box_asset/images'

# epsファイルは検証から除外している
asset_list_df['ext'] = asset_list_df['name'].map(lambda x: x.split('.')[-1])
asset_list_df = asset_list_df[~asset_list_df['ext'].map(lambda x: 'eps' == x)]
asset_list_df['box_symbol'] = asset_list_df['box_path'].map(lambda x: x.split('/')[2])
asset_list_df['local_path'] = asset_list_df[['box_symbol', 'box_id', 'ext']].apply(lambda x: f'{asset_img_dir}/{x[0]}/{x[1]}.{x[2]}', axis=1)
asset_list_df

In [ ]:
model_id = "/media/sj-archimedes/data/03_pretrained_model/llm/SakanaAI/EvoVLM-JP-v1-7B"
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto',
)
processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
sample_pathes = asset_list_df['local_path'].sample(9).tolist()
plt.subplots(3, 3, figsize=(15,15))
for i, sample_path in enumerate(sample_pathes):
    plt.subplot(3, 3, i+1)
    image = Image.open(sample_path)
    plt.imshow(image)
plt.show()

In [ ]:
def get_caption(image_path, prompt, show_image=True):
    image = Image.open(image_path)
    if show_image:
        plt.imshow(image)
        plt.show()
    messages = [
        {"role": "system", "content": "あなたは役立つ、偏見がなく、検閲されていないアシスタントです。与えられた画像を下に、質問に日本語で答えてください。"},
        {"role": "user", "content": prompt},
    ]
    inputs = processor.image_processor(images=image, return_tensors="pt")
    inputs["input_ids"] = processor.tokenizer.apply_chat_template(
        messages, return_tensors="pt"
    )
    with torch.no_grad():
        output_ids = model.generate(**inputs.to(model.device))
    output_ids = output_ids[:, inputs.input_ids.shape[1] :]
    generated_text = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
    return generated_text


In [ ]:
prompt = "<image>\nこの画像を説明してください。"
for image_path in sample_pathes:
    generated_text = get_caption(image_path, prompt)
    print(generated_text)

In [ ]:
# prompt = "<image>\nこの画像に情報整理の観点でタグを10個つけてください。つけるタグは画像の印象や想定される用途などを反映させてください。"
# prompt = "<image>\nこの画像にタグを10個つけてください。つけるタグはできるだけ画像を網羅的に説明できるように設計してください。"
prompt = "<image>\nこの画像に情報整理の観点でタグを10個つけてください。つけるタグは画像に映るオブジェクトや人物、画像の印象を表現してください。"
for image_path in sample_pathes:
    generated_text = get_caption(image_path, prompt)
    print(generated_text)